### Import Librarires And Dataset

In [ ]:
import warnings
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

#some settings to show data
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

# target_url = ("http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data")
target_url = ("datasets/abalone.data")
abalone = pd.read_csv(target_url)
abalone.columns = ['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight',
                   'Shell weight', 'Rings']

### Analyze the Data

In [ ]:
abalone.head()

In [ ]:
abalone.describe()

In [ ]:
abalone.info()

**Get target value**

In [ ]:
y = abalone["Rings"]
X = abalone.drop(columns="Rings")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
def is_numeric(x):
    return x == 'int' or x == 'float'
    # return int(x == 'int' or x == 'float')

In [ ]:
def is_numeric_v2(value):
    return isinstance(value, int) or isinstance(value, float)

In [ ]:
attribute_types = X_train.dtypes.apply(is_numeric)
attribute_types

In [ ]:
def build_dt(X, y, attribute_types, options=False):
    print("any implemantation")

In [ ]:
build_dt(X_train, y_train, attribute_types)